In [1]:
import pandas as pd

df = pd.read_csv(
    r"C:\Users\Kavya\OneDrive\Documents\TFL\DATAAnalysis\data\tfl_arrivals.csv",
    header=None)

In [2]:
print(df.head())

                                                  0           1   2    3   \
0  Tfl.Api.Presentation.Entities.Prediction, Tfl.... -1016672935   1  225   
1  Tfl.Api.Presentation.Entities.Prediction, Tfl....  1775985613   1  241   
2  Tfl.Api.Presentation.Entities.Prediction, Tfl....  1715108579   1  274   
3  Tfl.Api.Presentation.Entities.Prediction, Tfl.... -1864629153   1  270   
4  Tfl.Api.Presentation.Entities.Prediction, Tfl.... -1864432545   1  240   

            4                                  5         6         7   \
0  940GZZLUFPK  Finsbury Park Underground Station  victoria  Victoria   
1  940GZZLUVIC       Victoria Underground Station  victoria  Victoria   
2  940GZZLUGPK     Green Park Underground Station  victoria  Victoria   
3  940GZZLUGPK     Green Park Underground Station  victoria  Victoria   
4  940GZZLUGPK     Green Park Underground Station  victoria  Victoria   

                        8         9   ...           11  \
0  Southbound - Platform 4   inbound  ..

In [4]:
print(df.shape)

(2686, 21)


In [5]:
print("Duplicate rows:", df.duplicated().sum())


Duplicate rows: 0


In [6]:
print("null_values:", df.isnull().sum())

null_values: 0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9     1108
10    1969
11       0
12       0
13       0
14       0
15       0
16       0
17       0
18       0
19       0
20     134
dtype: int64


In [7]:
column_names = [
    "$type",
    "id",
    "operationType",
    "vehicleId",
    "naptanId",
    "stationName",
    "lineId",
    "lineName",
    "platformName",
    "direction",
    "bearing",
    "destinationNaptanId",
    "destinationName",
    "timestamp",
    "timeToStation",
    "currentLocation",
    "towards",
    "expectedArrival",
    "timeToLive",
    "modeName",
    "timing"
]


In [8]:
df.columns = column_names


In [9]:
print(df.head())
print(df.columns)


                                               $type          id  \
0  Tfl.Api.Presentation.Entities.Prediction, Tfl.... -1016672935   
1  Tfl.Api.Presentation.Entities.Prediction, Tfl....  1775985613   
2  Tfl.Api.Presentation.Entities.Prediction, Tfl....  1715108579   
3  Tfl.Api.Presentation.Entities.Prediction, Tfl.... -1864629153   
4  Tfl.Api.Presentation.Entities.Prediction, Tfl.... -1864432545   

   operationType  vehicleId     naptanId                        stationName  \
0              1        225  940GZZLUFPK  Finsbury Park Underground Station   
1              1        241  940GZZLUVIC       Victoria Underground Station   
2              1        274  940GZZLUGPK     Green Park Underground Station   
3              1        270  940GZZLUGPK     Green Park Underground Station   
4              1        240  940GZZLUGPK     Green Park Underground Station   

     lineId  lineName             platformName direction  ...  \
0  victoria  Victoria  Southbound - Platform 4   in

In [10]:
print("null_values:", df.isnull().sum())

null_values: $type                     0
id                        0
operationType             0
vehicleId                 0
naptanId                  0
stationName               0
lineId                    0
lineName                  0
platformName              0
direction              1108
bearing                1969
destinationNaptanId       0
destinationName           0
timestamp                 0
timeToStation             0
currentLocation           0
towards                   0
expectedArrival           0
timeToLive                0
modeName                  0
timing                  134
dtype: int64


In [11]:
#Check FULL duplicate rows

full_duplicates = df[df.duplicated()]

print("Full duplicate rows:", full_duplicates.shape[0])


Full duplicate rows: 0


**Check logical duplicates**
1. same train with same arrival time / depature time appears mutiple times only timestamp changes- still it counted as no duplicate becuase of time stamp.
2. so ["id", "vehicleId", "expectedArrival"] are unique 


In [ ]:
#subset--checks only important columns ; keep=False -- shows all copies, not just one

keep=False → shows all copies, not just one
logical_duplicates = df[df.duplicated(
    subset=["id", "vehicleId", "expectedArrival"],
    keep=False
)]

print("Logical duplicate rows:", logical_duplicates.shape[0])


Logical duplicate rows: 1228


In [13]:
#view those logical duplicates 

logical_duplicates[
    ["stationName", "vehicleId", "expectedArrival", "timestamp"]
].sort_values(["vehicleId", "expectedArrival"])


,stationName,vehicleId,expectedArrival,timestamp
620,King's Cross St. Pancras Underground Station,202,2025-11-26T16:45:08Z,2025-11-26T16:44:06.8338759Z
886,King's Cross St. Pancras Underground Station,202,2025-11-26T16:45:08Z,2025-11-26T16:45:06.1352808Z
795,Euston Underground Station,202,2025-11-26T16:46:32Z,2025-11-26T16:45:06.1352808Z
1128,Euston Underground Station,202,2025-11-26T16:46:32Z,2025-11-26T16:45:08.8907362Z
1454,Euston Underground Station,202,2025-11-26T16:46:32Z,2025-11-26T16:46:11.2987753Z
...,...,...,...,...
2205,Victoria Underground Station,277,2025-11-26T17:05:42Z,1036
1743,Vauxhall Underground Station,277,2025-11-26T17:09:41Z,1337
2111,Vauxhall Underground Station,277,2025-11-26T17:09:41Z,1275
1647,Stockwell Underground Station,277,2025-11-26T17:10:42Z,1398


In [14]:
#Read all 6 CSV files

FILES = {
    "Central": "data/Central.csv",
    "Northern": "data/Northern.csv",
    "Victoria": "data/Victoria.csv",
    "Piccadilly": "data/Piccadilly.csv",
    "Metropolitan": "data/Metropolitan.csv",
    "Bakerloo": "data/Bakerloo.csv"
}

dataframes = {}

for line, path in FILES.items():
    df = pd.read_csv(path)  
    df["source_file"] = line
    dataframes[line] = df

print("All files loaded successfully")


All files loaded successfully


In [16]:
print("COLUMN COUNT CHECK")

for line, df in dataframes.items():
    print(f"{line}: {len(df.columns)} columns")


COLUMN COUNT CHECK
Central: 28 columns
Northern: 28 columns
Victoria: 28 columns
Piccadilly: 28 columns
Metropolitan: 28 columns
Bakerloo: 28 columns


In [15]:
#count check 

print("ROW COUNT CHECK")
for line, df in dataframes.items():
    print(f"{line}: {df.shape[0]} rows")


ROW COUNT CHECK
Central: 1473 rows
Northern: 1923 rows
Victoria: 691 rows
Piccadilly: 1370 rows
Metropolitan: 499 rows
Bakerloo: 524 rows


In [17]:
print("ALL COLUMN NAMES PER FILE")

for line, df in dataframes.items():
    print(f"\n {line} ({len(df.columns)} columns):")
    for col in df.columns:
        print(col)


ALL COLUMN NAMES PER FILE

 Central (28 columns):
$type
id
operationType
vehicleId
naptanId
stationName
lineId
lineName
platformName
direction
bearing
destinationNaptanId
destinationName
timestamp
timeToStation
currentLocation
towards
expectedArrival
timeToLive
modeName
timing.$type
timing.countdownServerAdjustment
timing.source
timing.insert
timing.read
timing.sent
timing.received
source_file

 Northern (28 columns):
$type
id
operationType
vehicleId
naptanId
stationName
lineId
lineName
platformName
direction
bearing
destinationNaptanId
destinationName
timestamp
timeToStation
currentLocation
towards
expectedArrival
timeToLive
modeName
timing.$type
timing.countdownServerAdjustment
timing.source
timing.insert
timing.read
timing.sent
timing.received
source_file

 Victoria (28 columns):
$type
id
operationType
vehicleId
naptanId
stationName
lineId
lineName
platformName
direction
bearing
destinationNaptanId
destinationName
timestamp
timeToStation
currentLocation
towards
expectedArrival
timeT

In [18]:
all_columns = set()

for df in dataframes.values():
    all_columns.update(df.columns)

print("UNIQUE COLUMNS ACROSS ALL FILES")
for col in sorted(all_columns):
    print(col)


UNIQUE COLUMNS ACROSS ALL FILES
$type
bearing
currentLocation
destinationName
destinationNaptanId
direction
expectedArrival
id
lineId
lineName
modeName
naptanId
operationType
platformName
source_file
stationName
timeToLive
timeToStation
timestamp
timing.$type
timing.countdownServerAdjustment
timing.insert
timing.read
timing.received
timing.sent
timing.source
towards
vehicleId


In [21]:
# check null values 

print("NULL VALUE ANALYSIS")

for line, df in dataframes.items():
    print(f"\n {line}")
    print("Total null values:", df.isnull().sum().sum())



NULL VALUE ANALYSIS

 Central
Total null values: 1543

 Northern
Total null values: 2025

 Victoria
Total null values: 771

 Piccadilly
Total null values: 1488

 Metropolitan
Total null values: 980

 Bakerloo
Total null values: 623


In [24]:
#perdataset per coloumn - null value count

print("NULL VALUES PER COLUMN (PER DATASET)\n")

for line, df in dataframes.items():
    print(f"DATASET: {line}")
    
    null_counts = df.isnull().sum()
    
    # Print only columns that actually have nulls
    null_counts = null_counts[null_counts > 0]
    
    if null_counts.empty:
        print("No null values found")
    else:
        for col, count in null_counts.items():
            print(f"{col} : {count}")
    
    print("-" * 50)


NULL VALUES PER COLUMN (PER DATASET)

DATASET: Central
direction : 70
bearing : 1473
--------------------------------------------------
DATASET: Northern
direction : 100
bearing : 1923
currentLocation : 2
--------------------------------------------------
DATASET: Victoria
direction : 80
bearing : 691
--------------------------------------------------
DATASET: Piccadilly
direction : 57
bearing : 1370
destinationNaptanId : 12
destinationName : 12
currentLocation : 37
--------------------------------------------------
DATASET: Metropolitan
bearing : 499
currentLocation : 19
direction : 190
destinationNaptanId : 136
destinationName : 136
--------------------------------------------------
DATASET: Bakerloo
direction : 81
bearing : 524
destinationNaptanId : 9
destinationName : 9
--------------------------------------------------


In [25]:
# check duplicates per dataset

print("FULL DUPLICATE CHECK (PER DATASET)")

for line, df in dataframes.items():
    full_dup_count = df.duplicated().sum()
    print(f"{line}: {full_dup_count} full duplicate rows")


FULL DUPLICATE CHECK (PER DATASET)
Central: 0 full duplicate rows
Northern: 0 full duplicate rows
Victoria: 0 full duplicate rows
Piccadilly: 0 full duplicate rows
Metropolitan: 0 full duplicate rows
Bakerloo: 0 full duplicate rows


In [26]:
# check logical duplicates
#Logical duplicate = same train, same station, same arrival time

print("LOGICAL DUPLICATE CHECK (PER DATASET)")

for line, df in dataframes.items():
    logical_dup_count = df.duplicated(
        subset=["vehicleId", "stationName", "expectedArrival"]
    ).sum()

    print(f"{line}: {logical_dup_count} logical duplicates")



LOGICAL DUPLICATE CHECK (PER DATASET)
Central: 10 logical duplicates
Northern: 6 logical duplicates
Victoria: 3 logical duplicates
Piccadilly: 6 logical duplicates
Metropolitan: 4 logical duplicates
Bakerloo: 1 logical duplicates
